In [1]:
from model.pointpillars import PointPillars

In [2]:
import torch
from torch import nn
from model.pointpillars import PointPillars
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [37]:
# Define the fully connected layer
class FCModel(nn.Module):
      def __init__(self, original_model):
        super(FCModel, self).__init__()
        self.pointpillars = original_model
        self.fc = nn.Linear(256 * 62 * 54, 512)  # Fully connected layer to reduce to 512 dimensions

      def forward(self, x):
          # Get features from the PointPillars backbone
          features = self.pointpillars(x)  # Assuming output shape is [B, 256, 62, 54]
          
          if isinstance(features, list):
            features = features[-1]  # Use the last feature tensor if multiple outputs exist
          # Flatten features
          
          features_flat = features.view(features.size(0), -1)  # Flatten to [B, 256 * 62 * 54]
          
          # Pass through the FC layer
          reduced_features = self.fc(features_flat)  # Output shape is [B, 512]
          return reduced_features


In [38]:
# Assuming `model` is the PointPillars model
pretrained_model_path = r"C:\Users\hussa\OneDrive\Desktop\Projects\ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data\poinpillar\pre_trained\epoch_160.pth"
model_state_dict = torch.load(pretrained_model_path, map_location=torch.device('cpu'))

# Initialize the PointPillars model
model = PointPillars(
    nclasses=3,
    voxel_size=[0.16, 0.16, 4],
    point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
    max_num_points=32,
    max_voxels=(16000, 40000)  # Training and inference max_voxels
)

# Load the filtered state dict into the model (without the head weights)
model.load_state_dict(model_state_dict, strict=False)

# # Now replace the head layer with Identity

model.neck = nn.Identity()
model.head = nn.Identity()

model = FCModel(model)

# Set the model to evaluation mode
model.eval()


FCModel(
  (pointpillars): PointPillars(
    (pillar_layer): PillarLayer(
      (voxel_layer): Voxelization(
        (voxelizer): HardVoxelization()
      )
    )
    (pillar_encoder): PillarEncoder(
      (conv): Conv1d(9, 64, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (backbone): Backbone(
      (multi_blocks): ModuleList(
        (0): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (5): ReLU(inplace=True)
          (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      

In [ ]:
# Function to load and preprocess PCD file
def load_pcd_file(pcd_path):
    # Load the point cloud data using Open3D
    pcd = o3d.io.read_point_cloud(pcd_path)

    # Convert points to numpy array
    points = np.asarray(pcd.points)  # (N, 3)

    # Check if colors are available and add them
    if pcd.has_colors():
        colors = np.asarray(pcd.colors)  # (N, 3)
        data = np.hstack((points, colors))  # Combine points and colors
    else:
        data = points  # Only points available
        
    return data


# Preprocess the point cloud data for the model
def preprocess_point_cloud(pcd_points):

    # For example, voxel size (you may need to tune this)
    voxel_size = 0.2
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcd_points)
    downsampled_pcd = pcd.voxel_down_sample(voxel_size)

    # Convert the downsampled points to numpy array
    downsampled_points = np.asarray(downsampled_pcd.points)  # Shape: (N', 3)

    # Convert to torch tensor and add batch dimension
    input_tensor = torch.tensor(downsampled_points, dtype=torch.float32).unsqueeze(0)

    return input_tensor


In [44]:
# Inference function
def run_inference(input_tensor):
    with torch.no_grad():
        # Run the model inference (assuming the model returns features directly)
        features = model(input_tensor)
    return features

In [45]:
# Example usage
pcd_path = r"C:\Users\hussa\OneDrive\Desktop\Projects\Project\MOT\INFRA-3DRC_scene-15\INFRA-3DRC_scene-15\lidar_01\lidar_01__data\lidar_01__2023-06-02-21-28-09-321.pcd"
points = load_pcd_file(pcd_path)

x_min, x_max = np.min(points[:, 0]), np.max(points[:, 0])
y_min, y_max = np.min(points[:, 1]), np.max(points[:, 1])

print(x_min, x_max, y_min, y_max)
x_range = x_max - x_min
y_range = y_max - y_min

print(x_range, y_range)

# Preprocess the point cloud data
input_tensor = preprocess_point_cloud(points)

# Run inference
features = run_inference(input_tensor)

print(features.shape)
print(f'fea {features}')


-153.81289672851562 121.6452865600586 -66.90157318115234 96.09734344482422
275.4581832885742 162.99891662597656
torch.Size([1, 512])
fea tensor([[-2.4810,  0.8540, -0.4522, -0.3311,  0.8102, -0.0539, -0.3551, -1.9374,
          0.0669,  0.4492, -0.4861, -1.0186,  0.5423, -0.5881, -1.2853, -1.0903,
         -0.9193, -0.6869, -0.3095, -0.1703, -1.3830,  0.7142,  0.1527, -0.0593,
          2.0985,  1.8965,  0.0577,  0.3120, -0.5745, -2.0948,  1.4861, -1.2225,
          0.9504, -0.6405,  0.9987, -0.2844,  0.5083,  0.7006,  0.0780,  0.3686,
         -0.6501, -0.0199, -0.6352,  0.7376, -1.1079, -1.2099, -0.9091,  0.8616,
         -0.1441, -1.2520, -0.0346, -0.1112, -1.0732, -0.0260,  1.2822,  0.6861,
         -0.3417,  0.9892, -0.9693,  0.8202, -0.9028, -0.8242, -0.4553, -1.3038,
          1.0806,  0.6932,  0.3632,  0.6487,  0.4570,  0.4616,  0.6220,  0.7195,
          0.8832, -0.6363, -0.8894,  0.5026,  1.3928,  0.1092,  0.3461,  0.3314,
         -0.5511,  0.3914, -0.6886,  0.3008,  0.0445,